In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

#### Don't copy code, make a function!!

In [2]:
def pd_to_tf(dataframe):
    return tf.data.Dataset.from_tensor_slices((tf.cast(dataframe['text'].values, tf.string), tf.cast(dataframe['fake'].values, tf.int32)))

# Train/Val/Test Split

In [3]:
# load data from json
df = pd.read_json('../data/classification.json', lines=True).drop('index', axis=1)

# shuffle rows
df = df.sample(frac=1, random_state=1816)

# manual train/val-test split
# training set
df_train = df.iloc[:int(df.shape[0]*.9)]
# training fold of training set
df_train = df_train.iloc[:int(df_train.shape[0]*.9)]
# validation fold of training set
df_val = df_train.iloc[int(df_train.shape[0]*.9):]
# test set
df_test = df.iloc[int(df.shape[0]*.9):]

## Convert pandas.DataFrame to tensorflow.Dataset

In [4]:
train_data = pd_to_tf(df_train)
validation_data = pd_to_tf(df_val)
test_data = pd_to_tf(df_test)

In [35]:
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1'
# embedding = 'https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1' # 30s per epoch
# embedding = 'https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1' # errors: model probably too big to load
# embedding = 'https://tfhub.dev/google/universal-sentence-encoder-lite/2' # error
# embedding = 'https://tfhub.dev/google/universal-sentence-encoder/4' # error: probably too big

In [36]:
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Identity: Dst tensor is not initialized. [Op:Identity]

In [ ]:
model = tf.keras.Sequential([hub_layer, tf.keras.layers.Dense(16, activation='relu'), tf.keras.layers.Dense(1)])

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_4 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [25]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Train for 95 steps, validate for 10 steps
Epoch 1/20
95/95 [==============================] - 28s 296ms/step - loss: 0.5762 - accuracy: 0.6799 - val_loss: 0.4041 - val_accuracy: 0.8365
Epoch 2/20
95/95 [==============================] - 27s 286ms/step - loss: 0.3346 - accuracy: 0.8553 - val_loss: 0.2523 - val_accuracy: 0.8892
Epoch 3/20
95/95 [==============================] - 27s 286ms/step - loss: 0.2531 - accuracy: 0.8905 - val_loss: 0.2045 - val_accuracy: 0.9117
Epoch 4/20
95/95 [==============================] - 27s 289ms/step - loss: 0.2163 - accuracy: 0.9076 - val_loss: 0.1748 - val_accuracy: 0.9285
Epoch 5/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1926 - accuracy: 0.9187 - val_loss: 0.1546 - val_accuracy: 0.9368
Epoch 6/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1751 - accuracy: 0.9268 - val_loss: 0.1411 - val_accuracy: 0.9444
Epoch 7/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1619 - accuracy: 0.9330

gnews-swivel-20dim/1

Train for 95 steps, validate for 10 steps
Epoch 1/20
95/95 [==============================] - 3s 35ms/step - loss: 0.6591 - accuracy: 0.5925 - val_loss: 0.5575 - val_accuracy: 0.6907
Epoch 2/20
95/95 [==============================] - 1s 15ms/step - loss: 0.4910 - accuracy: 0.7546 - val_loss: 0.4238 - val_accuracy: 0.8021
Epoch 3/20
95/95 [==============================] - 1s 15ms/step - loss: 0.3897 - accuracy: 0.8220 - val_loss: 0.3516 - val_accuracy: 0.8433
Epoch 4/20
95/95 [==============================] - 1s 15ms/step - loss: 0.3370 - accuracy: 0.8500 - val_loss: 0.3131 - val_accuracy: 0.8587
Epoch 5/20
95/95 [==============================] - 1s 15ms/step - loss: 0.3060 - accuracy: 0.8647 - val_loss: 0.2878 - val_accuracy: 0.8793
Epoch 6/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2849 - accuracy: 0.8746 - val_loss: 0.2699 - val_accuracy: 0.8859
Epoch 7/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2692 - accuracy: 0.8816 - val_loss: 0.2557 - val_accuracy: 0.8935
Epoch 8/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2570 - accuracy: 0.8879 - val_loss: 0.2446 - val_accuracy: 0.8979
Epoch 9/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2473 - accuracy: 0.8923 - val_loss: 0.2350 - val_accuracy: 0.8999
Epoch 10/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2390 - accuracy: 0.8967 - val_loss: 0.2280 - val_accuracy: 0.9049
Epoch 11/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2318 - accuracy: 0.8999 - val_loss: 0.2207 - val_accuracy: 0.9073
Epoch 12/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2257 - accuracy: 0.9022 - val_loss: 0.2145 - val_accuracy: 0.9098
Epoch 13/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2199 - accuracy: 0.9050 - val_loss: 0.2087 - val_accuracy: 0.9127
Epoch 14/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2150 - accuracy: 0.9075 - val_loss: 0.2042 - val_accuracy: 0.9150
Epoch 15/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2104 - accuracy: 0.9093 - val_loss: 0.1989 - val_accuracy: 0.9174
Epoch 16/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2063 - accuracy: 0.9116 - val_loss: 0.1952 - val_accuracy: 0.9205
Epoch 17/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2022 - accuracy: 0.9131 - val_loss: 0.1914 - val_accuracy: 0.9240
Epoch 18/20
95/95 [==============================] - 1s 15ms/step - loss: 0.1982 - accuracy: 0.9156 - val_loss: 0.1867 - val_accuracy: 0.9217
Epoch 19/20
95/95 [==============================] - 1s 15ms/step - loss: 0.1952 - accuracy: 0.9169 - val_loss: 0.1841 - val_accuracy: 0.9217
Epoch 20/20
95/95 [==============================] - 1s 15ms/step - loss: 0.1916 - accuracy: 0.9174 - val_loss: 0.1798 - val_accuracy: 0.9263

gnews-swivel-20dim-with-oov/1

Train for 95 steps, validate for 10 steps
Epoch 1/20
95/95 [==============================] - 2s 23ms/step - loss: 0.6693 - accuracy: 0.5871 - val_loss: 0.5567 - val_accuracy: 0.7057
Epoch 2/20
95/95 [==============================] - 1s 15ms/step - loss: 0.4968 - accuracy: 0.7483 - val_loss: 0.4298 - val_accuracy: 0.7945
Epoch 3/20
95/95 [==============================] - 1s 15ms/step - loss: 0.3933 - accuracy: 0.8203 - val_loss: 0.3504 - val_accuracy: 0.8462
Epoch 4/20
95/95 [==============================] - 1s 15ms/step - loss: 0.3341 - accuracy: 0.8510 - val_loss: 0.3058 - val_accuracy: 0.8684
Epoch 5/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2998 - accuracy: 0.8680 - val_loss: 0.2785 - val_accuracy: 0.8826
Epoch 6/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2772 - accuracy: 0.8787 - val_loss: 0.2585 - val_accuracy: 0.8878
Epoch 7/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2608 - accuracy: 0.8863 - val_loss: 0.2446 - val_accuracy: 0.8954
Epoch 8/20
95/95 [==============================] - ETA: 0s - loss: 0.2486 - accuracy: 0.89 - 1s 15ms/step - loss: 0.2486 - accuracy: 0.8914 - val_loss: 0.2333 - val_accuracy: 0.9018
Epoch 9/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2385 - accuracy: 0.8957 - val_loss: 0.2251 - val_accuracy: 0.9022
Epoch 10/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2305 - accuracy: 0.8990 - val_loss: 0.2180 - val_accuracy: 0.9047
Epoch 11/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2234 - accuracy: 0.9031 - val_loss: 0.2104 - val_accuracy: 0.9125
Epoch 12/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2169 - accuracy: 0.9064 - val_loss: 0.2047 - val_accuracy: 0.9131
Epoch 13/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2115 - accuracy: 0.9084 - val_loss: 0.1997 - val_accuracy: 0.9197
Epoch 14/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2063 - accuracy: 0.9110 - val_loss: 0.1949 - val_accuracy: 0.9203
Epoch 15/20
95/95 [==============================] - 1s 15ms/step - loss: 0.2017 - accuracy: 0.9128 - val_loss: 0.1905 - val_accuracy: 0.9199
Epoch 16/20
95/95 [==============================] - 1s 15ms/step - loss: 0.1973 - accuracy: 0.9156 - val_loss: 0.1862 - val_accuracy: 0.9244
Epoch 17/20
95/95 [==============================] - 1s 15ms/step - loss: 0.1932 - accuracy: 0.9166 - val_loss: 0.1824 - val_accuracy: 0.9277
Epoch 18/20
95/95 [==============================] - 1s 15ms/step - loss: 0.1902 - accuracy: 0.9183 - val_loss: 0.1786 - val_accuracy: 0.9269
Epoch 19/20
95/95 [==============================] - 1s 15ms/step - loss: 0.1863 - accuracy: 0.9205 - val_loss: 0.1754 - val_accuracy: 0.9304
Epoch 20/20
95/95 [==============================] - 1s 15ms/step - loss: 0.1828 - accuracy: 0.9219 - val_loss: 0.1731 - val_accuracy: 0.9275

nnlm-en-dim50/1

Train for 95 steps, validate for 10 steps
Epoch 1/20
95/95 [==============================] - 28s 296ms/step - loss: 0.5762 - accuracy: 0.6799 - val_loss: 0.4041 - val_accuracy: 0.8365
Epoch 2/20
95/95 [==============================] - 27s 286ms/step - loss: 0.3346 - accuracy: 0.8553 - val_loss: 0.2523 - val_accuracy: 0.8892
Epoch 3/20
95/95 [==============================] - 27s 286ms/step - loss: 0.2531 - accuracy: 0.8905 - val_loss: 0.2045 - val_accuracy: 0.9117
Epoch 4/20
95/95 [==============================] - 27s 289ms/step - loss: 0.2163 - accuracy: 0.9076 - val_loss: 0.1748 - val_accuracy: 0.9285
Epoch 5/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1926 - accuracy: 0.9187 - val_loss: 0.1546 - val_accuracy: 0.9368
Epoch 6/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1751 - accuracy: 0.9268 - val_loss: 0.1411 - val_accuracy: 0.9444
Epoch 7/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1619 - accuracy: 0.9330 - val_loss: 0.1281 - val_accuracy: 0.9460
Epoch 8/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1522 - accuracy: 0.9370 - val_loss: 0.1215 - val_accuracy: 0.9578
Epoch 9/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1439 - accuracy: 0.9405 - val_loss: 0.1108 - val_accuracy: 0.9574
Epoch 10/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1367 - accuracy: 0.9447 - val_loss: 0.1049 - val_accuracy: 0.9607
Epoch 11/20
95/95 [==============================] - 27s 289ms/step - loss: 0.1304 - accuracy: 0.9467 - val_loss: 0.1003 - val_accuracy: 0.9627
Epoch 12/20
95/95 [==============================] - 28s 293ms/step - loss: 0.1255 - accuracy: 0.9490 - val_loss: 0.0954 - val_accuracy: 0.9611
Epoch 13/20
95/95 [==============================] - 29s 307ms/step - loss: 0.1209 - accuracy: 0.9512 - val_loss: 0.0912 - val_accuracy: 0.9660
Epoch 14/20
95/95 [==============================] - 30s 319ms/step - loss: 0.1166 - accuracy: 0.9516 - val_loss: 0.0864 - val_accuracy: 0.9660
Epoch 15/20
95/95 [==============================] - 34s 355ms/step - loss: 0.1137 - accuracy: 0.9538 - val_loss: 0.0840 - val_accuracy: 0.9683
Epoch 16/20
95/95 [==============================] - 34s 357ms/step - loss: 0.1101 - accuracy: 0.9552 - val_loss: 0.0827 - val_accuracy: 0.9642
Epoch 17/20
95/95 [==============================] - 34s 356ms/step - loss: 0.1070 - accuracy: 0.9556 - val_loss: 0.0790 - val_accuracy: 0.9691
Epoch 18/20
95/95 [==============================] - 34s 357ms/step - loss: 0.1037 - accuracy: 0.9571 - val_loss: 0.0767 - val_accuracy: 0.9716
Epoch 19/20
95/95 [==============================] - 34s 357ms/step - loss: 0.1005 - accuracy: 0.9584 - val_loss: 0.0737 - val_accuracy: 0.9738
Epoch 20/20
95/95 [==============================] - 35s 368ms/step - loss: 0.0975 - accuracy: 0.9596 - val_loss: 0.0718 - val_accuracy: 0.9708

# Test Set Metrics

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

12/12 - 0s - loss: 0.3069 - accuracy: 0.8752
loss: 0.307
accuracy: 0.875
